In [1]:
import fdapdepy as fdapde
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation

# mesh
nodes = np.loadtxt("unit_square/points.txt", dtype=float)
triangles = np.loadtxt("unit_square/elements.txt", dtype=int)
triangles = triangles - 1 # sempre stessa storia
boundary = np.loadtxt("unit_square/boundary.txt", dtype=int)
nodes.shape

mesh = fdapde.domain(nodes, triangles, boundary)
basis = fdapde.functional_basis(mesh, 1)


In [2]:
pde_params = {"diff": 1.}

In [3]:
# pde_type = 1 simple laplacian
# -lapl(u) = f
#  u = 0 
# u_ex = sin(pi x) sin(pi y)
u_ex = np.sin(np.pi * mesh.nodes()[:,0]) * np.sin(np.pi*mesh.nodes()[:,1])

In [4]:
pde_type = 1
_pde = fdapde.cpp_pde_2_2_1(mesh, 1, pde_params)

In [5]:
# set dirichlet bc (è inutile in questo caso)
_pde.set_dirichlet_bc(u_ex)

In [6]:
# set forcing
quad_nodes = _pde.get_quadrature_nodes()
forcing = (1+np.pi*np.pi) * np.sin(np.pi * quad_nodes[:,0]) * np.sin(np.pi*quad_nodes[:,1])
_pde.set_forcing(forcing)

In [7]:
# init
_pde.init()

In [8]:
# compute solution
from scipy import sparse
print(_pde.stiff().shape)
print(_pde.force().shape)

(10201, 10201)
(10201, 1)


In [11]:
# set dirichlet bc
stiff = sparse.csr_matrix(_pde.stiff())
rhs = _pde.force()
for i in range(0, (len(boundary)-1)):
    stiff[i,:] = 0. * stiff[i,:]
    stiff[i,i] = 1.
    rhs[i] = u_ex[i]

In [12]:
sol = sparse.linalg.spsolve(stiff, rhs)

print("err_l_inf = ", np.max(np.abs(sol - u_ex))) # :)

err_l_inf =  5.462518053803086e-08
